In [ ]:
%env DSP_CACHEBOOL=False

In [ ]:
%run ./module-00-data-prep.ipynb
train = train
val = val
test = test
len(train), len(val), len(test)

## Demo with GPT4-o

In [ ]:
import logging
import sys

logging.disable(sys.maxsize)
MODEL = "gpt-4o-ai-factory"
API_BASE = "https://aifactorydemo8563613420.openai.azure.com/openai/deployments/gpt-4o-ai-factory/chat/completions?api-version=2023-03-15-preview"
API_VERSION = "2023-03-15-preview"

In [ ]:
import dspy

lm = dspy.AzureOpenAI(
    api_base=API_BASE,
    api_version=API_VERSION,
    deployment_id=MODEL,
)
dspy.configure(lm=lm)
dspy.settings.configure(lm=lm)

In [ ]:
class SMSClassifierNaiveSignature(dspy.Signature):
    """
    Given an SMS text, predict whether it is ham, spam, or smishing.
    Output only the predicted label.
    """

    text = dspy.InputField(desc="The SMS text to classify")
    label = dspy.OutputField(desc="The predicted label")

In [ ]:
classifier = dspy.Predict(SMSClassifierNaiveSignature)

In [ ]:
from sms_classifier import SMSClassifier

sms_classifier = SMSClassifier(lm)

In [ ]:
from datetime import datetime

from evaluation_helpers import validate_answer
from langfuse_extensions import EvaluateWithLangfuse

TIMESTAMP = f"{int(datetime.now().timestamp())}"
session = f"Run-{MODEL}-{TIMESTAMP}"
print(session)
# from evaluation_helpers import EvaluateWithLangfuse
evaluator = EvaluateWithLangfuse(
    devset=test, num_threads=1, display_progress=True, run_id=session
)
evaluator(sms_classifier, metric=validate_answer)

In [ ]:
from evaluation_helpers import calculate_metrics, fetch_traces
from sms_classifier import Label

classes = [l.value for l in Label]

In [ ]:
# Run-gpt-4o-ai-factory-1729432239
traces_gpt_4 = fetch_traces(run_id="Run-gpt-4o-ai-factory-1729432239")
metrics_gpt_4 = calculate_metrics(traces_gpt_4, classes)
metrics_gpt_4["macro"]